# Week 1 — Orientation & Setup

Goal: Ensure environment and data access are ready, and document key columns.

Checklist
- [ ] Read the project Task Prompt and objectives
- [ ] Confirm Python/Git/IDE installed and accessible
- [ ] Verify access to CRMLS CSV datasets in this folder
- [ ] Deliverable: Short notes describing key columns

Instructions
- Run the cells in this Week 1 section first.
- If any check fails, add notes here and we’ll fix it before proceeding.


In [7]:
# Week 1: Environment check
import sys, subprocess
import shutil
import platform

print({
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "git": shutil.which("git") is not None,
})

# IDE check (heuristic): running in Jupyter implies OK
print({"jupyter": True})


{'python': '3.12.4', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'git': True}
{'jupyter': True}


In [8]:
# Week 1: Dataset access confirmation
from pathlib import Path
import re

DATA_DIR = Path.cwd()
RAW_PATTERN = re.compile(r"^CRMLSSold\d{6}.*\.csv$", re.I)
raw_files = sorted([p for p in DATA_DIR.iterdir() if p.is_file() and RAW_PATTERN.match(p.name)])

print(f"Found {len(raw_files)} CSV files")
raw_files[:5]


Found 10 CSV files


[PosixPath('/Users/anvith/Downloads/Datasets IDX/CRMLSSold202501_filled.csv'),
 PosixPath('/Users/anvith/Downloads/Datasets IDX/CRMLSSold202502_filled.csv'),
 PosixPath('/Users/anvith/Downloads/Datasets IDX/CRMLSSold202503_filled.csv'),
 PosixPath('/Users/anvith/Downloads/Datasets IDX/CRMLSSold202504_filled.csv'),
 PosixPath('/Users/anvith/Downloads/Datasets IDX/CRMLSSold202505_filled.csv')]

In [9]:
# Peek first file for columns
import pandas as pd

if not raw_files:
    raise SystemExit("No CRMLS CSVs found. Place files like CRMLSSoldYYYYMM_*.csv in this folder.")

sample_path = raw_files[0]
df_sample = pd.read_csv(sample_path, nrows=200, dtype=str, low_memory=False)
df_sample.columns = [c.strip() for c in df_sample.columns]
print("Sample file:", sample_path.name)
print("Columns:", len(df_sample.columns))
df_sample.columns[:50]


Sample file: CRMLSSold202501_filled.csv
Columns: 80


Index(['BuyerAgentAOR', 'ListAgentAOR', 'Flooring', 'ViewYN', 'WaterfrontYN',
       'BasementYN', 'PoolPrivateYN', 'OriginalListPrice', 'ListingKey',
       'ListAgentEmail', 'CloseDate', 'ClosePrice', 'ListAgentFirstName',
       'ListAgentLastName', 'Latitude', 'Longitude', 'UnparsedAddress',
       'PropertyType', 'LivingArea', 'ListPrice', 'DaysOnMarket',
       'ListOfficeName', 'BuyerOfficeName', 'CoListOfficeName',
       'ListAgentFullName', 'CoListAgentFirstName', 'CoListAgentLastName',
       'BuyerAgentMlsId', 'BuyerAgentFirstName', 'BuyerAgentLastName',
       'FireplacesTotal', 'AssociationFeeFrequency', 'AboveGradeFinishedArea',
       'ListingKeyNumeric', 'MLSAreaMajor', 'TaxAnnualAmount',
       'CountyOrParish', 'MlsStatus', 'ElementarySchool', 'AttachedGarageYN',
       'ParkingTotal', 'BuilderName', 'PropertySubType', 'LotSizeAcres',
       'SubdivisionName', 'BuyerOfficeAOR', 'YearBuilt', 'StreetNumberNumeric',
       'ListingId', 'BathroomsTotalInteger'],
      dt

In [10]:
# Week 1 Deliverable: Short notes for key columns

# Map likely important columns to human-friendly descriptions.
# Adjust as needed if your schema differs.
key_columns = {
    "ClosePrice": "Final sales price at closing; modeling target.",
    "CloseDate": "Date the sale closed; used for time-aware splits and trend features.",
    "LivingArea": "Interior living area (sq ft). Used for PPSF and size.",
    "BedroomsTotal": "Number of bedrooms.",
    "BathroomsTotalInteger": "Number of bathrooms (integer).",
    "LotSizeAcres": "Lot size in acres.",
    "YearBuilt": "Year the property was built.",
    "GarageSpaces": "Number of garage spaces.",
    "PoolPrivateYN": "Whether the property has a private pool.",
    "Latitude": "Latitude coordinate for geo features.",
    "Longitude": "Longitude coordinate for geo features.",
    "PostalCode": "ZIP/postal code for neighborhood aggregation.",
    "City": "City name.",
    "CountyOrParish": "County for geographic grouping.",
    "ElementarySchool": "Assigned elementary school/district indicator.",
}

# Try to match with actual columns (case-insensitive substring contains)
actual_cols = list(df_sample.columns)
matched = {}
unmatched = []
for k, note in key_columns.items():
    m = next((c for c in actual_cols if k.lower() == c.lower() or k.lower() in c.lower()), None)
    if m:
        matched[m] = note
    else:
        unmatched.append(k)

print("Matched columns (notes):")
for c, note in matched.items():
    print(f"- {c}: {note}")

if unmatched:
    print("\nColumns not found (review and adjust names if needed):")
    for k in unmatched:
        print(f"- {k}")


Matched columns (notes):
- ClosePrice: Final sales price at closing; modeling target.
- CloseDate: Date the sale closed; used for time-aware splits and trend features.
- LivingArea: Interior living area (sq ft). Used for PPSF and size.
- BedroomsTotal: Number of bedrooms.
- BathroomsTotalInteger: Number of bathrooms (integer).
- LotSizeAcres: Lot size in acres.
- YearBuilt: Year the property was built.
- GarageSpaces: Number of garage spaces.
- PoolPrivateYN: Whether the property has a private pool.
- Latitude: Latitude coordinate for geo features.
- Longitude: Longitude coordinate for geo features.
- PostalCode: ZIP/postal code for neighborhood aggregation.
- City: City name.
- CountyOrParish: County for geographic grouping.
- ElementarySchool: Assigned elementary school/district indicator.


If any of the above key columns are missing or named differently, please provide your schema or an example row so I can refine the mapping precisely (e.g., `ClosePrice` is `ClosePrice` vs `closeprice`).


# CRMLS Close Price Prediction

This notebook builds an end-to-end, leakage-safe model to predict property close price from CRMLS historical data.

- Loads only the original CSVs in this folder
- Cleans and combines monthly files
- Excludes listing-only fields (ListPrice, DaysOnMarket, agents, marketing) to avoid leakage
- Engineers neighborhood-level signals (median price per sqft using only prior sales)
- Trains and evaluates a baseline gradient boosting model
- Provides a lightweight prediction function for new properties

Notes:
- All work is performed in-notebook; no external artifacts are written.
- Replace the sample prediction with a real property when available.


In [ ]:
# Imports
import os
from pathlib import Path
import re
import json
from typing import List, Optional, Dict

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

DATA_DIR = Path.cwd()
RAW_PATTERN = re.compile(r"^CRMLSSold\d{6}.*\.csv$", re.I)

pd.set_option("display.max_columns", 200)
np.set_printoptions(suppress=True)


In [ ]:
# Discover and load raw CSVs
raw_files = sorted([p for p in DATA_DIR.iterdir() if p.is_file() and RAW_PATTERN.match(p.name)])
if not raw_files:
    raise SystemExit("No CRMLS CSV files found in the current directory.")

raw_files


In [ ]:
def standardize_column_names(columns: List[str]) -> List[str]:
    out = []
    seen = {}
    for col in columns:
        c = col.strip()
        c = re.sub(r"\s+", "_", c)
        c = c.replace("/", "_").replace("\\", "_")
        c = c.replace("(", "").replace(")", "")
        c = c.replace("%", "pct").replace("#", "num").replace("$", "usd")
        c = c.replace("-", "_").replace(".", "_")
        c = re.sub(r"[^0-9A-Za-z_]+", "", c)
        c = re.sub(r"_+", "_", c).strip("_").lower()
        if not c:
            c = "col"
        if c in seen:
            seen[c] += 1
            c = f"{c}_{seen[c]}"
        else:
            seen[c] = 0
        out.append(c)
    return out


def load_and_clean(path: Path) -> pd.DataFrame:
    # read as strings first; let later steps cast
    df = pd.read_csv(path, dtype=str, low_memory=False)
    df.columns = standardize_column_names(list(df.columns))
    # drop repeated header rows
    header_tuple = tuple(df.columns)
    mask = df.apply(lambda r: tuple(r.astype(str)) == header_tuple, axis=1)
    if mask.any():
        df = df.loc[~mask].copy()
    # trim strings and set common NAs
    obj_cols = df.select_dtypes(include=["object"]).columns
    for c in obj_cols:
        s = df[c].astype("string").str.strip()
        df[c] = s.replace({"": pd.NA, "na": pd.NA, "n/a": pd.NA, "none": pd.NA})
    return df


dfs = [load_and_clean(p) for p in raw_files]
len(dfs), [df.shape for df in dfs]


In [ ]:
# Combine and basic info
combined = pd.concat(dfs, axis=0, ignore_index=True)
combined = combined.drop_duplicates()
print(combined.shape)
combined.head(3)


In [ ]:
# Detect target and date columns
TARGET_CANDS = ["close_price","closeprice","sold_price","soldprice","sale_price","saleprice","final_sales_price","finalsalesprice","final_price","finalprice","closing_price","closingprice"]
DATE_CANDS = ["close_date","closedate","sold_date","solddate","closing_date","closingdate","recording_date"]

cols_lower = {c.lower(): c for c in combined.columns}

def detect_col(cands):
    for c in cands:
        if c in cols_lower:
            return cols_lower[c]
    for c in cands:
        for col in combined.columns:
            if c in col.lower():
                return col
    return None

TARGET = detect_col(TARGET_CANDS)
DATE_COL = detect_col(DATE_CANDS)
TARGET, DATE_COL


In [ ]:
# Remove leakage-prone columns and select property/location features
LEAKAGE_KEYS = ["listprice","originallistprice","original_list_price","list_price","listing","dom","days_on_market","daysonmarket","cdom","status","agent","broker","office","remarks","marketing","virtual","photo","syndication","license","member","mls"]
PROP_KEYS = ["bed","bath","living_area","sqft","square_feet","lot","lot_size","acres","year_built","yearbuilt","stories","units","pool","spa","garage","parking","hoa","zoning"]
LOC_KEYS  = ["latitude","longitude","lat","lon","lng","zip","zipcode","postal","postalcode","city","county","neighborhood","school","district"]

feature_cols = []
for c in combined.columns:
    if c == TARGET: continue
    low = c.lower()
    if any(k in low for k in LEAKAGE_KEYS):
        continue
    if any(k in low for k in PROP_KEYS) or any(k in low for k in LOC_KEYS) or (DATE_COL and c == DATE_COL):
        feature_cols.append(c)

len(feature_cols), feature_cols[:20]


In [ ]:
# Cast types and engineer neighborhood PPSF features

def to_numeric_clean(s: pd.Series) -> pd.Series:
    if s.dtype.kind in "biufc":
        return s
    x = s.astype(str)
    x = x.str.replace(",", "", regex=False).str.replace("$", "", regex=False).str.replace("%", "", regex=False)
    return pd.to_numeric(x, errors="coerce")

work = combined[feature_cols + ([TARGET] if TARGET else [])].copy()

# target numeric
if TARGET is None:
    raise SystemExit("Could not detect target close price column.")
work[TARGET] = to_numeric_clean(work[TARGET])

# likely area column
AREA_CANDS = ["living_area","livingarea","sqft","square_feet","square_footage"]
area_col = next((c for c in AREA_CANDS if c in work.columns), None)
if area_col is not None:
    area = to_numeric_clean(work[area_col])
    work["ppsf_target"] = work[TARGET] / area

# date column
if DATE_COL and DATE_COL in work.columns:
    work[DATE_COL] = pd.to_datetime(work[DATE_COL], errors="coerce")

# zip/postal
zip_col = next((c for c in ["zip","zipcode","postal","postalcode"] if c in work.columns), None)

# rolling by count (prior N sales)
if zip_col is not None and "ppsf_target" in work.columns:
    tmp = work[[zip_col, DATE_COL] if DATE_COL else [zip_col]].copy()
    if DATE_COL:
        tmp = tmp.join(work[["ppsf_target"]])
        tmp = tmp.sort_values([zip_col, DATE_COL])
    else:
        tmp = tmp.join(work[["ppsf_target"]])
        tmp = tmp.sort_values([zip_col]).reset_index(drop=True)
    def grp_roll(g):
        return g["ppsf_target"].rolling(window=200, min_periods=10).median().shift(1)
    tmp["zip_median_ppsf_prev"] = tmp.groupby(zip_col, group_keys=False).apply(grp_roll)
    work["zip_median_ppsf_prev"] = tmp["zip_median_ppsf_prev"].values

# time-window (90D) prior median if dates available
if zip_col is not None and DATE_COL and "ppsf_target" in work.columns:
    g = work[[zip_col, DATE_COL, "ppsf_target"]].dropna(subset=[DATE_COL]).copy()
    g = g.sort_values([zip_col, DATE_COL])
    def time_roll(h):
        s = h.set_index(DATE_COL)["ppsf_target"]
        return s.rolling("90D", min_periods=10).median().shift(1)
    rolled = g.groupby(zip_col, group_keys=False).apply(time_roll).rename("zip_median_ppsf_prev_90d").reset_index()
    work = work.merge(rolled, how="left", on=[zip_col, DATE_COL])

work.head(3)


In [ ]:
# Prepare dataset for modeling
# Drop rows without target
model_df = work.dropna(subset=[TARGET]).copy()

X = model_df.drop(columns=[TARGET])
y = pd.to_numeric(model_df[TARGET], errors="coerce")
mask = y.notna()
X = X.loc[mask]
y = y.loc[mask]

# Identify numeric vs categorical
num_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
cat_cols = [c for c in X.columns if c not in num_cols]

len(X), len(num_cols), len(cat_cols)


In [ ]:
# Time-aware split if possible
if DATE_COL and DATE_COL in X.columns:
    dates = pd.to_datetime(X[DATE_COL], errors="coerce")
    order = dates.argsort(kind="mergesort")
    X_sorted = X.iloc[order]
    y_sorted = y.iloc[order]
    split_idx = int(0.8 * len(X_sorted))
    X_train, X_test = X_sorted.iloc[:split_idx], X_sorted.iloc[split_idx:]
    y_train, y_test = y_sorted.iloc[:split_idx], y_sorted.iloc[split_idx:]
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


In [ ]:
# Build pipeline
numeric_transformer = SimpleImputer(strategy="median")
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", min_frequency=20)),
])

preprocess = ColumnTransformer(
    transformers=[
      ("num", numeric_transformer, [c for c in X_train.columns if pd.api.types.is_numeric_dtype(X_train[c])]),
      ("cat", categorical_transformer, [c for c in X_train.columns if not pd.api.types.is_numeric_dtype(X_train[c])]),
    ],
    remainder="drop",
    sparse_threshold=0.3,
)

model = HistGradientBoostingRegressor(
    max_depth=None,
    learning_rate=0.08,
    max_iter=600,
    min_samples_leaf=25,
    l2_regularization=0.05,
    random_state=42,
)

pipe = Pipeline(steps=[("prep", preprocess), ("model", model)])
pipe


In [ ]:
# Train and evaluate
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = float(np.mean(np.abs((y_test - y_pred) / np.clip(np.abs(y_test), 1e-9, None))))
rmse = float(np.sqrt(np.mean((y_test - y_pred) ** 2)))

print({"R2": r2, "MAE": mae, "MAPE": mape, "RMSE": rmse})


## Quick EDA snippets

Use these as needed to inspect the data shape, missingness, and distributions.


In [ ]:
# EDA cells (optional to run)
combined.info()

# Missingness top 20
(combined.isna().sum().sort_values(ascending=False).head(20))

# Numeric summary
combined.select_dtypes(include=["number"]).describe().T.head(15)


## Prediction helper

Provide a property dictionary and get a predicted close price. Only property and location features are used; listing-only fields are ignored.


In [ ]:
def predict_close_price(pipe, feature_cols: List[str], sample: Dict) -> float:
    X = pd.DataFrame([sample])
    for c in feature_cols:
        if c not in X.columns:
            X[c] = np.nan
    X = X[feature_cols]
    return float(pipe.predict(X)[0])

# Example usage (edit values as appropriate):
sample = {
    "beds": 3,
    "bathroomstotalinteger": 2,
    "livingarea": 1500,
    "lotsizeacres": 0.12,
    "zipcode": "92880",
    "yearbuilt": 1985,
    "latitude": 33.9,
    "longitude": -117.6,
}

try:
    pred = predict_close_price(pipe, list(X_train.columns), sample)
    print(f"Predicted close price: ${pred:,.0f}")
except Exception as e:
    print("Prediction failed; adjust feature names/values:", e)
